### CA2: Classification
#### Hadi Heidari Rad

In [8]:
import pandas as pd

df = pd.read_excel('dataset.xls') # dependency: xlrd
df.drop('Unnamed: 0', inplace=True, axis=1)
df.head()

,Customer,Agent,SalesAgentEmailID,ContactEmailID,Stage,Product,Close_Value,Created Date,Close Date
0,Konex,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255.0,2016-01-04,2016-05-24
1,Finjob,Rosalina Dieter,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45.0,2016-01-04,2016-11-02
2,Kinnamplus,Donn Cantrell,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,NaN,2016-01-06,2016-12-07
3,Genco Pura Olive Oil Company,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054.0,2016-01-06,2016-09-11
4,Stanredtax,Lajuana Vencill,lajuana@piedpiper.com,candice@stanredtax.com,Won,MG Advanced,3180.0,2016-01-07,2016-01-31


Now we should find out what columns contain NaN and then replace them with a value:

In [33]:
df.isna().any()

Customer             False
Agent                False
SalesAgentEmailID    False
ContactEmailID       False
Stage                False
Product              False
Close_Value          False
Created Date         False
Close Date           False
dtype: bool

In [34]:
df['Close_Value'].fillna(df['Close_Value'].mean(), inplace=True)
df.head()

,Customer,Agent,SalesAgentEmailID,ContactEmailID,Stage,Product,Close_Value,Created Date,Close Date
0,Konex,Corliss Cosme,corliss@piedpiper.com,delila@konex.com,Won,GTX Plus Basic,1255.000000,2016-01-04,2016-05-24
1,Finjob,Rosalina Dieter,rosalina@piedpiper.com,belinda@finjob.com,Won,MG Special,45.000000,2016-01-04,2016-11-02
2,Kinnamplus,Donn Cantrell,donn@piedpiper.com,monte@kinnamplus.com,Lost,MG Special,1852.915504,2016-01-06,2016-12-07
3,Genco Pura Olive Oil Company,James Ascencio,james@piedpiper.com,karole@gencopuraoliveoilcompany.com,In Progress,MG Advanced,1054.000000,2016-01-06,2016-09-11
4,Stanredtax,Lajuana Vencill,lajuana@piedpiper.com,candice@stanredtax.com,Won,MG Advanced,3180.000000,2016-01-07,2016-01-31


Now splitting the dataframe into two sections:
- train data (the ones whose stage attribute is either 'won' or 'lost')
- test data (the ones whose stage attribute is 'in progress')

In [22]:
train_data, test_data = [x for _, x in df.groupby(df['Stage'] == 'In Progress')]